In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Forecasting using Prophet

### Installation

In [ ]:
!pip install statsmodels --quiet
!pip install fbprophet --quiet

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from fbprophet import Prophet

In [ ]:
cd ..

In [ ]:
ls

### Data preparation

In [ ]:
df = pd.read_csv('input/arsenal-data/prem-arsenal.csv')
df_1 = df.iloc[:38,:]
df_2 = df.iloc[38:2*(38),:]
df_3 = df.iloc[2*(38):3*(38):]
df.columns

In [ ]:
df_1['cum_pts'] = df_1.points.cumsum()
df_2['cum_pts'] = df_2.points.cumsum()
df_3['cum_pts'] = df_3.points.cumsum()

In [ ]:
df_table = pd.DataFrame()
df_table = df_table.append(df_1)
df_table = df_table.append(df_2)
df_table = df_table.append(df_3)
df_table = pd.get_dummies(df_table, columns=['referee'])

In [ ]:
df_s = df_table.copy()
df_s['date_col'] = pd.to_datetime(df_s['date_col'])
df_s = df_s.set_index('date_col')
df_s.index = pd.date_range(start='15/08/2018', periods=df_s.shape[0], freq='W')
s=seasonal_decompose(df_s['cum_pts'])

In [ ]:
s.plot()

### Modelling

#### Without regressors

In [ ]:
df_t = df_table[['date_col','cum_pts',]]
df_t.columns = ["ds", "y"]

In [ ]:
df_table.index = df_table['date_col']
df_table['cum_pts'].plot(figsize=(10, 8))

In [ ]:
df_table.columns

In [ ]:
m = Prophet(seasonality_mode='multiplicative')
m.fit(df_t)

In [ ]:
pred_df = pd.DataFrame()
pred_df['ds'] = pd.date_range(start='13/08/2021', periods=5, freq='W')


In [ ]:
preds = m.predict(pred_df)
preds['yhat'] = [int(x) for x in preds['yhat']]

In [ ]:
pred_df['cum_pts_preds'] = preds['yhat']
pred_df.head()

In [ ]:
pred_df.index = pred_df['ds']
pred_df['cum_pts_preds'].plot()

#### with home-advantage

In [ ]:
df_t = df_table[['date_col','cum_pts', 'home-adv']]
df_t.columns = ["ds", "y", "home-adv"]

In [ ]:
df_table.index = df_table['date_col']
df_table['cum_pts'].plot(figsize=(10, 8))

In [ ]:
m = Prophet(seasonality_mode='multiplicative')
m.add_regressor('home-adv')
m.fit(df_t)

In [ ]:
import random
my_list = [0]*19 + [1]*19
random.shuffle(my_list)

In [ ]:
pred_df = pd.DataFrame()
pred_df['ds'] = pd.date_range(start='13/08/2021', periods=38, freq='W')
pred_df['home-adv'] = my_list

In [ ]:
preds = m.predict(pred_df)
preds['yhat'] = [int(x) for x in preds['yhat']]

pred_df['cum_pts_preds'] = preds['yhat']
pred_df.tail()

In [ ]:
pred_df.index = pred_df['ds']
pred_df['cum_pts_preds'].plot()

#### with referees

In [ ]:
df_table.columns

In [ ]:
df_t = df_table[['date_col','cum_pts', 'home-adv', 'referee_Anthony Taylor',
                 'referee_Jonathan Moss', 'referee_Martin Atkinson',
                 'referee_Michael Oliver', 'referee_Mike Dean']]

df_t.columns = ["ds", "y", "home-adv",'referee_Anthony Taylor', 
                'referee_Jonathan Moss','referee_Martin Atkinson', 
                'referee_Michael Oliver','referee_Mike Dean']

In [ ]:
m = Prophet(seasonality_mode='multiplicative')
m.add_regressor('home-adv')
m.add_regressor('referee_Anthony Taylor')
m.add_regressor('referee_Jonathan Moss')
m.add_regressor('referee_Martin Atkinson')
m.add_regressor('referee_Michael Oliver')
m.add_regressor('referee_Mike Dean')
m.fit(df_t)

In [ ]:
pred_df = pd.DataFrame()
pred_df['ds'] = pd.date_range(start='13/08/2021', periods=5, freq='W')
pred_df['home-adv'] = [0, 1, 0, 1, 1]
pred_df['referee_Anthony Taylor'] = [0, 1, 0, 0, 1]
pred_df['referee_Jonathan Moss'] = [0, 0, 1, 0, 0]
pred_df['referee_Martin Atkinson'] = [1, 0, 0, 0, 0]
pred_df['referee_Michael Oliver'] = [0, 0, 0, 1, 0]
pred_df['referee_Mike Dean'] = [0, 1, 0, 0, 0]

In [ ]:
preds = m.predict(pred_df)
preds['yhat'] = [int(x) for x in preds['yhat']]

pred_df['cum_pts_preds'] = preds['yhat']
pred_df.head()

In [ ]:
pred_df.index = pred_df['ds']
pred_df['cum_pts_preds'].plot()

## Validation data

In [ ]:
df_t = df_table[['date_col','cum_pts', 'home-adv']]
df_t.columns = ["ds", "y", "home-adv"]

dtrain, dvalid = df_t.iloc[:76,:], df_t.iloc[76:,:]
dtrain.index = dtrain['ds']
dtrain['y'].plot(figsize=(10, 8))

In [ ]:
m = Prophet(seasonality_mode='multiplicative')
m.add_regressor('home-adv')
m.fit(dtrain)

In [ ]:
preds = m.predict(dvalid[['ds','home-adv']])
preds['predictions'] = [int(x) for x in preds['yhat']]
dvalid['forecast'] = preds['predictions']

In [ ]:
dvalid.index = preds.index
preds['yhat'] = [abs(x) for x in preds['yhat']]
preds['yhat_lower'] = [abs(x) for x in preds['yhat_lower']]
preds['yhat_upper'] = [abs(x) for x in preds['yhat_upper']]
dvalid[['yhat_lower', 'yhat_upper', 'yhat']] = preds[['yhat_lower', 'yhat_upper', 'yhat']]

In [ ]:
dvalid['original'] = dvalid['y']
dvalid['prediction'] = dvalid['yhat']
ax = dvalid[['original', 'prediction']].plot(figsize=(10,8), 
                                colormap='jet',marker='.', markersize=10, 
                             title='Arsenal season 2020-21 Forecast vs original - single forecast')

ax.set_xlabel("Date")
ax.set_ylabel("Points")

## Rolling prediction

In [ ]:
dvalid.shape

In [ ]:
window = 7
pr_df = pd.DataFrame()

for i in range(0, int(dvalid.shape[0]/window)):
    roll_df = dvalid.iloc[i*(window):i*(window) + window,:]
    print(i)
    m1 = Prophet(seasonality_mode='multiplicative')
    m1.add_regressor('home-adv')
    m1.fit(dtrain)
    preds = m.predict(roll_df[['ds','home-adv']])
    dtrain = dtrain.append(roll_df)
    pr_df  = pr_df.append(preds)

In [ ]:
dvalid.iloc[:pr_df.shape[0],:].index = pr_df.index
pr_df['yhat'] = [abs(x) for x in pr_df['yhat']]
pr_df['yhat_lower'] = [abs(x) for x in pr_df['yhat_lower']]
pr_df['yhat_upper'] = [abs(x) for x in pr_df['yhat_upper']]
dfinalv = dvalid.iloc[:pr_df.shape[0],:]
pr_df.index = pr_df['ds']
dfinalv.index = pr_df.index
dfinalv[['yhat_lower', 'yhat_upper', 'yhat']] = pr_df[['yhat_lower', 'yhat_upper', 'yhat']]

In [ ]:
dfinalv['original'] = dfinalv['y']
dfinalv['prediction'] = dfinalv['yhat']
ax = dfinalv[['original', 'prediction']].plot(figsize=(10, 8), 
                            colormap='jet', 
                            marker='.', 
                            markersize=10, 
                            title='Arsenal season 2020-21 Forecast vs original - rolling forecast with window as a week')
ax.set_xlabel("Date")
ax.set_ylabel("Points")

In [ ]:
pr_df.head(2)

In [ ]:
dfinalv.head(2)